In [ ]:
#@title Stable Diffusion web-ui colab от [Meldoner](https://github.com/Meldoner)
%cd /content
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!wget -nc --content-disposition https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors -P /content/stable-diffusion-webui/embeddings
#@markdown **Выбор готовой модели**
model = "openjourneyv4" #@param ["openjourneyv4","sd2.1","sd1.5","abyss-orange-mix3","anythingv3","anythingv4.5","deliberate2","dreamlike","dreamshaper5","kenshi","protogenx5.8","hassaku","realistic-visionv2","corneos-7th-heaven-mix","pastel-mix","counterfeit-v2.5","rev-animated"]
#@markdown **ВАЖНО**: для некоторых моделей необходим **VAE**, выберите его в настройках\
#@markdown По умолчанию устанавливается [EasyNegative](https://civitai.com/models/7808/easynegative)

#@markdown **Нажми, если ты хочешь загрузить свою модель**
load_own_model = False #@param{type:"boolean"}
model_url = "Вставь ссылку на модель (ckpt/safetensors)" #@param {type:"string"}

#@markdown **Нажми, если ты хочешь использовать свой VAE**
load_own_vae = False #@param{type:"boolean"}
vae_url = "Вставь ссылку на VAE" #@param {type:"string"}

#@markdown **Нажми, если ты хочешь использовать свои embeddings**
load_own_embedding = False #@param{type:"boolean"}
embedding_url = "Вставь ссылку на embedding" #@param {type:"string"}

#@markdown **Нажми, если ты хочешь использовать свою LORA**
load_own_lora = False #@param{type:"boolean"}
lora_url = "Вставь ссылку на LORA" #@param {type:"string"}

#@markdown Вы можете загрузить несколько моделей(и остальное), просто перезапустите скрипт

#@markdown **Если ты хочешь, чтобы я добавил больше моделей, создай запрос [здесь](https://github.com/Meldoner/stable-diffusion-webui-colab/issues)**
model_urls = {
    'openjourneyv4': 'https://huggingface.co/prompthero/openjourney-v4/resolve/main/openjourney-v4.ckpt',
    'abyss-orange-mix3': ['https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors', 'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt'],
    'anythingv3': ['https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0-pruned.safetensors', 'https://huggingface.co/admruul/anything-v3.0/resolve/main/Anything-V3.0.vae.pt'],
    'anythingv4.5': ['https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.safetensors', 'https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt'],
    'deliberate2': 'https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors',
    'dreamlike': 'https://huggingface.co/dreamlike-art/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.safetensors',
    'dreamshaper5': 'https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_5_beta2_BakedVae_fp16.safetensors',
    'kenshi': 'https://huggingface.co/SweetLuna/Kenshi/resolve/main/KENSHI%2001/KENSHI01_baked.ckpt',
    'protogenx5.8': 'https://huggingface.co/darkstorm2150/Protogen_x5.8_Official_Release/resolve/main/ProtoGen_X5.8.safetensors',
    'sd2.1': 'https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors',
    'sd1.5': 'https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors',
    'hassaku': ['https://huggingface.co/sinkinai/hassaku-hentai/resolve/main/hassakuHentaiModel_v11.safetensors', 'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt'],
    'realistic-visionv2': ['https://huggingface.co/SG161222/Realistic_Vision_V2.0/resolve/main/Realistic_Vision_V2.0.safetensors', 'https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors'],
    'corneos-7th-heaven-mix': ['https://huggingface.co/dwarfbum/corneos7thHeavenMix_v2/resolve/main/corneos7thHeavenMix_v2.safetensors', 'https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt'],
    'pastel-mix': 'https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix.safetensors',
    'counterfeit-v2.5': ['https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_pruned.safetensors','https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt'],
    'rev-animated': ['https://huggingface.co/s6yx/ReV_Animated/resolve/main/rev_1.2.2/rev_1.2.2.safetensors','https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt']
}

if model in model_urls and not load_own_model:
    if isinstance(model_urls[model], list):
          !wget -nc --content-disposition {model_urls[model][0]} -P /content/stable-diffusion-webui/models/Stable-diffusion
          !wget -nc --content-disposition {model_urls[model][1]} -P /content/stable-diffusion-webui/models/VAE
    else:
        !wget -nc --content-disposition {model_urls[model]} -P /content/stable-diffusion-webui/models/Stable-diffusion

if load_own_model:
  !wget -nc --content-disposition {model_url} -P /content/stable-diffusion-webui/models/Stable-diffusion

if load_own_vae:
  !wget -nc --content-disposition {vae_url} -P /content/stable-diffusion-webui/models/VAE

if load_own_embedding:
  !wget -nc --content-disposition {embedding_url} -P /content/stable-diffusion-webui/embeddings

if load_own_lora:
  !wget -nc --content-disposition {lora_url} -P /content/stable-diffusion-webui/models/Lora

!pip install -q fastapi==0.90.1 -U
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.18 triton==2.0.0 -U
%cd stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --no-half-vae --xformers --gradio-queue --enable-insecure-extension-access" REQS_FILE="requirements.txt" python launch.py